In [2]:
import torch

torch.manual_seed(42)

# Preprocessing

In [3]:
import pandas

df = pandas.read_csv('s&p500.csv')
df = df.set_index('Date')
df

,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2014-10-31 00:00:00+00:00,36.393887,39.179268,23.973495,41.956585,36.026073,18.773333,68.768227,70.120003,40.179710,35.525333,...,91.401855,22.913128,162.714340,24.432884,62.614067,32.120022,42.933430,99.750992,73.750000,34.574112
2014-11-03 00:00:00+00:00,37.588364,39.795147,24.284258,41.817753,35.778130,18.799999,68.641075,69.910004,40.260685,35.714283,...,90.792976,23.197344,162.286133,24.637280,61.675262,31.837324,42.772045,99.006737,73.879997,35.163273
2014-11-04 00:00:00+00:00,36.980907,40.458393,24.106678,41.427670,36.042606,18.910000,68.793671,71.070000,40.042057,37.445198,...,90.725319,23.319159,158.646683,24.432884,61.196133,31.748985,43.070908,97.930672,72.779999,36.468380
2014-11-05 00:00:00+00:00,36.934883,40.032017,24.164398,41.348339,36.083923,18.906668,69.921150,71.370003,40.511711,38.027210,...,92.010750,23.265015,153.919647,24.885473,61.552242,32.146519,43.298035,99.526817,73.070000,36.608208
2014-11-06 00:00:00+00:00,38.076160,40.875298,24.233505,41.460724,36.108704,19.026667,70.565430,72.099998,40.665558,37.770222,...,93.950172,23.170267,153.037628,24.126284,62.322708,32.782562,43.692513,98.809441,73.879997,37.018387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-25 00:00:00+00:00,130.190002,13.150000,231.410004,187.850006,114.220001,105.300003,360.799988,483.720001,230.169998,56.560001,...,290.450012,31.799999,97.989998,64.459999,119.489998,130.419998,133.039993,102.349998,360.089996,180.009995
2024-10-28 00:00:00+00:00,131.539993,13.600000,233.399994,189.679993,114.070000,105.660004,361.320007,481.040009,230.139999,57.240002,...,292.119995,31.950001,98.620003,64.480003,118.900002,130.789993,134.860001,103.599998,363.579987,182.759995
2024-10-29 00:00:00+00:00,131.229996,13.820000,233.669998,189.449997,113.400002,104.800003,363.040009,485.390015,235.229996,56.320000,...,292.179993,30.879999,99.000000,63.340000,117.279999,129.889999,133.970001,103.879997,384.679993,181.270004


In [4]:
import torch

def preprocess(df):
    data = torch.from_numpy(df.to_numpy()).to(torch.float32)
    ratios = torch.zeros_like(data)
    # calculate daily return ratio
    for d in range(1, data.size(0)):
        ratios[d-1] = (data[d] - data[d-1]) / data[d-1]
    # skip the first day which cannot calculate daily return ratio
    # and round data size to nearest multiple of batch_size
    days_in_quarter = 64
    num_quarters = data.size(0) // days_in_quarter
    ratios = ratios[:num_quarters * days_in_quarter]
    # split into batches
    ratios = ratios.view(num_quarters, days_in_quarter, 472)
    # shuffle the batches
    ratios = ratios[torch.randperm(ratios.size(0))]
    return ratios

train_data = preprocess(df)
print(train_data.size())
print(train_data[0])

torch.Size([39, 64, 472])
tensor([[ 0.0143,  0.0706,  0.0366,  ...,  0.0272,  0.0560,  0.0401],
        [ 0.0059,  0.0426,  0.0161,  ...,  0.0096,  0.0255,  0.0035],
        [ 0.0119,  0.0007,  0.0147,  ...,  0.0149,  0.0009,  0.0036],
        ...,
        [ 0.0109,  0.0391,  0.0169,  ...,  0.0076,  0.0278,  0.0072],
        [ 0.0058, -0.0004, -0.0013,  ...,  0.0074, -0.0166, -0.0021],
        [-0.0098,  0.0076, -0.0146,  ...,  0.0014, -0.0295,  0.0049]])


# Model

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FeedForward(nn.Module):
    def __init__(self, hidden_size, expand_ratio, dropout):
        super(FeedForward, self).__init__()
        self.linear = nn.Linear(hidden_size, hidden_size * expand_ratio)
        self.linear2 = nn.Linear(hidden_size * expand_ratio, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout)
    
    def forward(self, x):
        x = self.linear(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.dropout(x)
        return x

class Attention(nn.Module):
    def __init__(self, d_model, num_heads, expand_ratio, dropout):
        super().__init__()
        self.mha = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, batch_first=True)
        self.ln1 = nn.LayerNorm(d_model)
        self.ln2 = nn.LayerNorm(d_model)
        self.ffn = FeedForward(hidden_size=d_model, expand_ratio=expand_ratio, dropout=dropout)

    def forward(self, x, attn_mask=None):
        x1, _ = self.mha(x, x, x, attn_mask=attn_mask, need_weights=False)
        x2 = self.ln1(x + x1)
        return self.ln2(self.ffn(x2) + x2)

class SpatialTemporalAttention(nn.Module):
    def __init__(self, d_model=32, num_heads=2, expand_ratio=1, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.input_proj = nn.Linear(1, d_model)
        # self.time_embedding = nn.Embedding(T, d_model)
        # self.stock_embedding = nn.Embedding(N, d_model)
        self.spatial_attn = Attention(d_model, num_heads, expand_ratio, dropout)
        self.temporal_attn = Attention(d_model, num_heads, expand_ratio, dropout)
        self.output_proj = nn.Linear(d_model, 1)
    
    def forward(self, x):
        T, N = x.size()
        # x = self.input_proj(x.view(T, N, 1)) + self.stock_embedding(torch.arange(N).unsqueeze(0).expand(T, N))
        x = self.input_proj(x.view(T, N, 1))
        # IDEA: Each spatial head takes in a different type of correlation matrix.
        # Like one takes in positive pearson's coefficnet and the other takes in negative
        x = self.spatial_attn(x).view(N, T, self.d_model)
        # x = self.time_embedding(torch.arange(T).unsqueeze(0).expand(N, T).to(x.device)) + x
        temporal_causal_mask = torch.triu(torch.ones((T, T), dtype=torch.bool), diagonal=1).expand(N * self.num_heads, T, T).to(x.device)
        x = self.temporal_attn(x, attn_mask=temporal_causal_mask)
        return self.output_proj(x).view(T, N)


# Training

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
model = SpatialTemporalAttention().to(device)
train_data = train_data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.MSELoss()

model.train()

num_epochs = 10

for epoch in range(num_epochs):
    optimizer.zero_grad()
    for batch in train_data:
        out = model(batch[:-1, :])
        loss = loss_fn(out, batch[1:, :])
        print(loss.item())
        loss.backward()
        optimizer.step()

0.44850534200668335
0.41363847255706787
0.3818199932575226
0.3510570228099823
0.3196580708026886
0.2919963002204895
0.26685649156570435
0.2392898052930832
0.21464751660823822
0.19080907106399536
0.16897812485694885
0.14613159000873566
0.1259884536266327
0.10874313116073608
0.0915217399597168
0.07531362771987915
0.06039062887430191
0.048031389713287354
0.03678424656391144
0.02748842164874077
0.01969699189066887
0.012882313691079617
0.008324654772877693
0.005044253543019295
0.003518564160913229
0.004125204868614674
0.005911575630307198
0.009015347808599472
0.013898148201406002
0.020476840436458588
0.028352364897727966
0.037264417856931686
0.048365358263254166
0.06042589247226715
0.07363482564687729
0.08753436803817749
0.10422005504369736
0.12074971199035645
0.13660098612308502
0.15301358699798584
0.16984713077545166
0.1846066564321518
0.19793197512626648
0.2123691737651825
0.2230416238307953
0.23013320565223694
0.2376268059015274
0.24214597046375275
0.2450677454471588
0.24516557157039642

In [19]:
torch.save(model.state_dict(), 'model.pth')